In [ ]:
import pandas as pd
import requests
import numpy as np
from geopy.distance import geodesic

In [3]:
df = pd.read_csv(r"C:\Users\mrice\OneDrive\Documents\Data Science - Callum\Projects\Financial Projects\london_house_prices\london_flats.csv")

In [ ]:
def get_tube_stop(lat, lon):
    url = "https://api.tfl.gov.uk/StopPoint"
    params = {
        "stopTypes": "NaptanMetroStation",  # Focus on Metro stations only (tube stations)
        "radius": 2000,  # Radius of 2 km (can be adjusted)
        "lat": lat,  
        "lon": lon, 
        "app_id": "e6b5f350beb041ce94c1c8793d718fb7",
        "app_key": "efebe289b99e49728f27b9342e3acd7a"
    }

    # Make the GET request with the parameters
    response = requests.get(url, params=params)

    # Check if the request was successful
    if response.status_code == 200:
        stations = response.json()
        if stations and "stopPoints" in stations:
            
            # initialize distance
            min_distance = int(2)
            nearest_station = None

            for stations in stations['stopPoints']:
                station_name = stations['commonName']
                station_lat = stations['lat']
                station_lon = stations['lon']

                station_location = (station_lat, station_lon)
                property_location = (lat, lon)
                distance = geodesic(property_location, station_location).km

                if distance < min_distance:
                    min_distance = distance
                    nearest_station = station_name

            return nearest_station, round(min_distance, 2)

    return None, None



In [ ]:
# Create an empty list to store results
tube_data = []

# Iterate over rows using itertuples() (which is much faster than apply)
for row in df.itertuples(index=False):
    closest_station, distance = get_tube_stop(row.latitude, row.longitude)
    tube_data.append((row.property_id, row.latitude, row.longitude, closest_station, distance))

# Convert the list to a DataFrame
tube_df = pd.DataFrame(tube_data, columns=['property_id', 'latitude', 'longitude', 'closest_station', 'distance'])

print(tube_df.head())




                        property_id   latitude  longitude  \
0  c03d82e0169b9646f9628c766a820a96  51.518926  -0.173068   
1  0eaca9c01a72e47b55656d34229fbbf1  51.359806  -0.201280   
2  0e91ce654b6fd59d84c47e7c66fcb6ad  51.564169  -0.100908   
3  6afcfb6a633839bcb514e31960abbe15  51.529622  -0.167885   
4  45112861947e9869c0cff0d2b041d13b  51.429787  -0.170103   

                               closest_station  distance  
0  Edgware Road (Bakerloo) Underground Station      0.25  
1                                         None       inf  
2            Finsbury Park Underground Station      0.41  
3          St. John's Wood Underground Station      0.69  
4         Tooting Broadway Underground Station      0.27  


In [6]:
tube_df.to_csv("stations.csv", index=False)